# Email Agent
- Authenticates users
  - Only then let user(s) into the inbox
  - Dynamic tools and prompts based on the authentication stage
- Checks "inbox"
  - email in tool
- Sends emails after getting human approval
  - Human-in-the-loop

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from dataclasses import dataclass

@dataclass
class EmailContext:
    email_address: str = "pytholic@example.com"
    password: str = "password123"

In [3]:
from langchain.agents import AgentState

class AuthenticatedState(AgentState):
    authenticated: bool

In [4]:
from langchain.tools import tool, ToolRuntime
from langgraph.types import Command
from langchain.messages import ToolMessage

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails"""
    return """
    Hi pytholic, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, John (john@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send an response email"""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(email: str, password: str, runtime: ToolRuntime) -> Command:
    """Authenticate the user with the given email and password"""
    if email == runtime.context.email_address and password == runtime.context.password:
        return Command(update={
            "authenticated": True, 
            "messages": [ToolMessage(
                "Successfully authenticated", 
                tool_call_id=runtime.tool_call_id)]
        })
    else:
        return Command(update={
            "authenticated": False,
            "messages": [ToolMessage(
                "Authentication failed", 
                tool_call_id=runtime.tool_call_id)]
        })

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from collections.abc import Callable

@wrap_model_call
def dynamic_tool_call(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Allow read inbox and send email tools only if user provides correct email and password"""
    
    authenticated = request.state.get("authenticated")
    
    if authenticated:
        tools = [check_inbox, send_email]
    else:
        tools = [authenticate]

    request = request.override(tools=tools)
    return handler(request)

In [6]:
from langchain.agents.middleware import dynamic_prompt

authenticated_prompt = "You are a helpful assistant that can check the inbox and send emails."
unauthenticated_prompt = "You are a helpful assistant that can authenticate users."

@dynamic_prompt
def dynamic_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on authentication status"""
    authenticated = request.state.get("authenticated")

    if authenticated:
        return authenticated_prompt
    else:
        return unauthenticated_prompt

In [7]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
)

agent = create_agent(
    model,
    tools=[authenticate, check_inbox, send_email],
    checkpointer=InMemorySaver(),
    state_schema=AuthenticatedState,
    context_schema=EmailContext,
    middleware=[
        dynamic_tool_call, 
        dynamic_prompt,
        HumanInTheLoopMiddleware(
            interrupt_on={
                "authenticate": False,
                "check_inbox": False,
                "send_email": True,
            })
        ]
    )

In [14]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    # {"messages": [HumanMessage(content="pytholic@example.com password123")]},
    # {"messages": [HumanMessage(content="Please check my inbox")]},
    # {"messages": [HumanMessage(content="Draft a response to John's email and send it")]},
    {"messages": [HumanMessage(content="Subject: Meetup, Body: Accept and draft a body for me")]},
    context=EmailContext(),
    config=config
)

print(response['messages'][-1].content)

In [19]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'That '
                                                                          'sounds '
                                                                          'great! '
                                                                          "I'd "
                                                                          'love '
                                                                          'to '
                                                                          'grab '
                                                                          'a '
                                                                          'coffee '
                   

In [15]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi John,

That sounds great! I'd love to grab a coffee next week. Let me know what day and time works best for you.

Best,
Pytholic


In [20]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}  # or "reject"
    ), 
    config=config # Same thread ID to resume the paused conversation
)

print(response["messages"][-1].content)

I've sent an email to John with the subject "Meetup" and the body:

"Hi John,

That sounds great! I'd love to grab a coffee next week. Let me know what day and time works best for you.

Best,
Pytholic"


In [21]:
pprint(response)

{'authenticated': True,
 'messages': [HumanMessage(content='draft 1', additional_kwargs={}, response_metadata={}, id='f7c1c817-83d2-4305-aaa6-6f7f576f1d15'),
              AIMessage(content=[{'type': 'text', 'text': "I can help with that. What would you like to draft? If you're trying to authenticate, please provide your email and password.", 'extras': {'signature': 'CuYDAXLI2nys3GR4eB+X9uT0MXBbN4k6SJH91VE/I7qet6szWzBoK3rTn+g4tmKYUiCglv21fkfy8QmZNBlBdF6cIyRtj/hQJ65iVF9uRtfrc+K4hdwinWNTF0pTNRGDw1I7BaTEDukGosgVG/2uGvAapi6xh/HBL7j6HDh1xTKRILiO9JjaYj+Z3ryS5tWUbbdPwVY4tN9G53QlDJXypf18FID9FeDDtMkEv5foiAFkNfuCCn74uJIqaMsYcgsu8m7ffm8QeuaJL/cSM3AC7tO/LQJdmILXClC/qMoNHEHbQddYGR70eHS1fOKxWBPtrcVx3J+51T5vd+5yq4XFzm86E4bIgNbtP/RMDIkiPpSgO9nkrLOAeHCMQN3ALwXE4e8558VYLJQJOPpau/S5Vfia9QfMaXvET9edvTd+d5cLZEIGWbzVWhfxxzG1xI39aLYtDT2AdmDwMSFIYiapF+s9L9LH5dirjdBcaIfOyVKAEmnc3nIkzAi0UcVwAK2F1Hm0ui8kH8wrI4BKs82uOcbBEJQ/1IMns1c3/9Cx7nR5YRugF5YxJKksPBRZQQAZHYaTTfyb4liXmVQ1l/98M/1Cm2ccJWfw0n6hLVZvlEZzhV7SYDeU9H